In [1]:
import numpy as np
import pandas as pd 
from datetime import datetime


In [2]:
df = pd.read_csv('wiki_data.csv')
df=df[df.columns[1:]]
nan_values = df[df.columns[:26]].isna().sum().sort_values()
columns_keeping = nan_values[nan_values<6000].index
df = df[columns_keeping]
df['Based on'] = df['Based on'].apply(lambda x: 0 if pd.isna(x) else 1)

c:\users\jacob\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (22,23,24,26,27,28,29,30,31,33,35,36,37,38,39,40,41,42,43,44,45,46,47,48,51,53,54,55,62,65,66,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,85,86,87,88,89,90,91,92,93,94,95,97,98,99,100,101,102,103,104,105,108,109,110,112,114,115,116,117,118,119,120,121,122,127,128,129,130,131,132,133,134,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,28

In [3]:
for col in df.columns[1:]:
    try:
        df[col] = df[col].str[3:-3]
    except:
        print(col)
        

Based on


In [4]:
df['Running time'] = df['Running time'].str.extract(r'(\d+)')
df['Running time'] = pd.to_numeric(df['Running time'])
df['Starring'] = df['Starring'].str.split(',')
df['Produced by'] = df['Produced by'].str.split(',')
df['Release date'] = df['Release date'].str.split('(') 


In [5]:
df['Release date length'] = df['Release date'].apply(lambda x:len(x) if type(x) == list else x)
df['Release date'] =  df['Release date'][df['Release date length']!=10]

In [6]:
 def clean_release(row):
        if type(row)==list:
            if len(row)==1:
                return row[0]
            else:
                return row[1][:-1]
        else:
            row
            
df['Release date']=df['Release date'].apply(clean_release).str.split(')').str[0]

In [7]:
def fix_dates (row):
    if pd.isna(row):
        return row
    else:
        try : 
            return datetime.strptime(row, "%Y-%m-%d")
        except:
            try:
                return datetime.strptime(row, "%B %d, %Y")
            except:
                return row
    

df['Release date'] = df['Release date'].apply(fix_dates)

In [8]:
df['Box office length'] = df['Box office'].str.split('million').apply(lambda x:len(x) if type(x) == list else x)
df['Box office'] = df['Box office'].str[1:]
df['Box office'] =  df['Box office'].str.split('million')
df = df[~((df['Box office length']==3) | (df['Box office length']==4))]

In [10]:
import re
def clean_box(row):
    if type(row)==list:
        if len(row)==1:
            string = re.split(' |\[', row[0])[0]
            string = string.replace(',','')
            return   string #row[0].split('[')[0]
        else:
            try: return float(row[0])*1000000
            except: return row
    else:
        return row
df['Box office'] = df['Box office'].apply(clean_box)

In [14]:
df.head()

,title,Running time,Directed by,Starring,Release date,Produced by,Distributed by,Music by,Edited by,Cinematography,...,Box office,Country,Budget,Written by,Productioncompanies,Screenplay by,Based on,Productioncompany,Release date length,Box office length
0,The Adventures of Ford Fairlane,102.0,Renny Harlin,"[Andrew Dice Clay', 'Wayne Newton', 'Priscil...",1990-07-11 00:00:00,"[Steve Perry', 'Joel Silver]",20th Century Fox,Yello,Michael Tronick,Oliver Wood,...,2.14e+07,United States,$20 million[2],NaN,NaN,"David Arnott', 'James Cappe', 'Daniel Waters",1,Silver Pictures,2.0,2.0
1,"After Dark, My Sweet",114.0,James Foley,"[Jason Patric', 'Rachel Ward', 'Bruce Dern',...",1990-05-17 00:00:00,[Ric KidneyRobert Redlin],Avenue Pictures,Maurice Jarre,Howard E. Smith,Mark Plummer,...,2.7e+06,United States,$6 million,NaN,NaN,James FoleyRobert Redlin,1,Avenue Pictures,5.0,2.0
2,Air America (film),113.0,Roger Spottiswoode,"[Mel Gibson', 'Robert Downey Jr.', 'Nancy Tr...",1990-08-10 00:00:00,[Daniel MelnickMario Kassar],TriStar Pictures,Charles Gross,John BloomLois Freeman-Fox,Roger Deakins,...,5.77e+07,United States,$30–35 million[1],NaN,Carolco PicturesIndieProd Company,"John Eskow', 'Richard Rush",1,NaN,2.0,2.0
3,Alice (1990 film),106.0,Woody Allen,"[Alec Baldwin', 'Blythe Danner', 'Judy Davis...",1990-12-25 00:00:00,[Robert Greenhut],Orion Pictures,NaN,Susan E. Morse,Carlo Di Palma,...,7331647,United States,$12 million[2],Woody Allen,NaN,NaN,0,NaN,2.0,1.0
4,Almost an Angel,95.0,John Cornell,"[Paul Hogan', 'Elias Koteas', 'Linda Kozlowski]",1990-12-19 00:00:00,[John Cornell],Paramount Pictures,Maurice Jarre,David Stiven,Russell Boyd,...,6939946,United States,$25 million[citation needed],Paul Hogan,NaN,NaN,0,NaN,1.0,1.0
